In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image

# Lokasi dataset
train_dir = r"D:\Data Slayer Olah\DataSlayer"
test_dir = r"D:\Data Slayer Olah\data\test"

# Augmentasi Data untuk Latihan
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Dataset Training dan Validation
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed=123
)

val_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    seed=123
)

# Bangun Model ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Bekukan sebagian besar layer, kecuali beberapa layer terakhir
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Tambahkan lapisan tambahan di atas model dasar
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

# Latih Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=callbacks
)

# Simpan Model
model_path = 'resnet50_fall_detection_optimized.h5'
model.save(model_path)
print(f"Model berhasil disimpan di {model_path}")

# Evaluasi Model
loss, accuracy = model.evaluate(val_dataset)
print(f"Akurasi Validasi: {accuracy * 100:.2f}%")

# Prediksi pada Dataset Uji
def load_and_preprocess_images(test_dir, target_size):
    test_images = []
    image_names = []

    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
            test_images.append(img_array)
            image_names.append(img_name)
        except Exception as e:
            print(f"Gagal memproses {img_name}: {e}")

    return np.array(test_images), image_names

# Load and preprocess test images
test_images, image_names = load_and_preprocess_images(test_dir, target_size=(150, 150))

# Prediksi
def generate_predictions(model, test_images, image_names, output_csv):
    predictions = model.predict(test_images)
    df = pd.DataFrame({
        'id': image_names,
        'label': predictions.flatten()
    })
    # Inversi label (1 = Fall, 0 = Non-Fall)
    df['label'] = df['label'].apply(lambda x: 0 if x > 0.5 else 1)
    df.to_csv(output_csv, index=False)
    print(f"Prediksi berhasil disimpan ke {output_csv}")

output_csv = 'test_predictions_optimized.csv'
generate_predictions(model, test_images, image_names, output_csv)


Found 1722 images belonging to 2 classes.
Found 430 images belonging to 2 classes.
Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.7517 - loss: 0.5432 - val_accuracy: 0.9465 - val_loss: 0.1884 - learning_rate: 1.0000e-04
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 51s 938ms/step - accuracy: 0.9446 - loss: 0.1703 - val_accuracy: 0.9605 - val_loss: 0.1495 - learning_rate: 1.0000e-04
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 49s 914ms/step - accuracy: 0.9436 - loss: 0.1461 - val_accuracy: 0.9628 - val_loss: 0.1580 - learning_rate: 1.0000e-04
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - accuracy: 0.9585 - loss: 0.1372 - val_accuracy: 0.9651 - val_loss: 0.1243 - learning_rate: 1.0000e-04
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.9466 - loss: 0.1444 - val_accuracy: 0.9674 - val_loss: 0.1226 - learning_rate: 1.0000e-04
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9649 - loss: 0.0952 - val_accuracy: 0.9558 - val_loss: 0.1697 - learning_rate

Model berhasil disimpan di resnet50_fall_detection_optimized.h5
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 617ms/step - accuracy: 0.9708 - loss: 0.0949
Akurasi Validasi: 96.98%
68/68 ━━━━━━━━━━━━━━━━━━━━ 47s 667ms/step
Prediksi berhasil disimpan ke test_predictions_optimized.csv


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image

# Lokasi dataset
train_dir = r"D:\Data Slayer Olah\DataSlayer"
test_dir = r"D:\Data Slayer Olah\data\test"

# Augmentasi Data untuk Latihan
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Dataset Training dan Validation
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed=123
)

val_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    seed=123
)

# Bangun Model EfficientNetV2S
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Bekukan sebagian besar layer, kecuali beberapa layer terakhir
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Tambahkan lapisan tambahan di atas model dasar
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

# Latih Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=callbacks
)

# Simpan Model
model_path = 'efficientnetv2s_fall_detection_optimized.h5'
model.save(model_path)
print(f"Model berhasil disimpan di {model_path}")

# Evaluasi Model
loss, accuracy = model.evaluate(val_dataset)
print(f"Akurasi Validasi: {accuracy * 100:.2f}%")

# Prediksi pada Dataset Uji
def load_and_preprocess_images(test_dir, target_size):
    test_images = []
    image_names = []

    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = tf.keras.applications.efficientnet_v2.preprocess_input(img_array)
            test_images.append(img_array)
            image_names.append(img_name)
        except Exception as e:
            print(f"Gagal memproses {img_name}: {e}")

    return np.array(test_images), image_names

# Load and preprocess test images
test_images, image_names = load_and_preprocess_images(test_dir, target_size=(150, 150))

# Prediksi
def generate_predictions(model, test_images, image_names, output_csv):
    predictions = model.predict(test_images)
    df = pd.DataFrame({
        'id': image_names,
        'label': predictions.flatten()
    })
    # Inversi label (1 = Fall, 0 = Non-Fall)
    df['label'] = df['label'].apply(lambda x: 0 if x > 0.5 else 1)
    df.to_csv(output_csv, index=False)
    print(f"Prediksi berhasil disimpan ke {output_csv}")

output_csv = 'test_predictions_efficienetv2s.csv'
generate_predictions(model, test_images, image_names, output_csv)


Found 1722 images belonging to 2 classes.
Found 430 images belonging to 2 classes.
Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 70s 910ms/step - accuracy: 0.6864 - loss: 0.5737 - val_accuracy: 0.9163 - val_loss: 0.2719 - learning_rate: 1.0000e-04
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 46s 849ms/step - accuracy: 0.8922 - loss: 0.2823 - val_accuracy: 0.9140 - val_loss: 0.2151 - learning_rate: 1.0000e-04
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9281 - loss: 0.2237 - val_accuracy: 0.9465 - val_loss: 0.1741 - learning_rate: 1.0000e-04
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9342 - loss: 0.2041 - val_accuracy: 0.9442 - val_loss: 0.1637 - learning_rate: 1.0000e-04
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 838ms/step - accuracy: 0.9337 - loss: 0.1949 - val_accuracy: 0.9465 - val_loss: 0.1404 - learning_rate: 1.0000e-04
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 840ms/step - accuracy: 0.9269 - loss: 0.1894 - val_accuracy: 0.9651 - val_loss: 0.1214 - learnin

Model berhasil disimpan di efficientnetv2s_fall_detection_optimized.h5
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 746ms/step - accuracy: 0.9643 - loss: 0.0961
Akurasi Validasi: 95.81%
68/68 ━━━━━━━━━━━━━━━━━━━━ 58s 774ms/step
Prediksi berhasil disimpan ke test_predictions_efficienetv2s.csv


In [ ]:
import pandas as pd

# Load the two CSV files
file1_path = 'path_to_file1.csv'  # Ganti dengan path file pertama
file2_path = 'path_to_file2.csv'  # Ganti dengan path file kedua

# Read the files into DataFrames
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Merge the two DataFrames on a common column (e.g., 'id')
comparison = pd.merge(df1, df2, on='id', how='outer', suffixes=('_file1', '_file2'))

# Find rows where there are differences in the 'label' column
differences = comparison[comparison['label_file1'] != comparison['label_file2']]

# Save differences to a new CSV file
output_path = 'differences.csv'
differences.to_csv(output_path, index=False)
print(f"Differences saved to {output_path}")

# Print summary of differences
print(f"Total differences: {len(differences)}")
print(differences)
